In [8]:
import pandas as pd
import numpy as np
import warnings
import scipy.stats as stats
warnings.filterwarnings('ignore')

In [3]:
data = pd.read_csv('gene_high_throughput_sequencing.csv')
print(data.shape)
data.head()

(72, 15750)


,Patient_id,Diagnosis,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,...,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
0,STT5425_Breast_001_normal,normal,1.257614,2.408148,13.368622,9.494779,20.880435,12.722017,9.494779,54.349694,...,4.761250,1.257614,1.257614,1.257614,1.257614,1.257614,23.268694,1.257614,1.257614,1.257614
1,STT5427_Breast_023_normal,normal,4.567931,16.602734,42.477752,25.562376,23.221137,11.622386,14.330573,72.445474,...,6.871902,1.815112,1.815112,1.815112,1.815112,1.815112,10.427023,1.815112,1.815112,1.815112
2,STT5430_Breast_002_normal,normal,2.077597,3.978294,12.863214,13.728915,14.543176,14.141907,6.232790,57.011005,...,7.096343,2.077597,2.077597,2.077597,2.077597,2.077597,22.344226,2.077597,2.077597,2.077597
3,STT5439_Breast_003_normal,normal,2.066576,8.520713,14.466035,7.823932,8.520713,2.066576,10.870009,53.292034,...,5.200770,2.066576,2.066576,2.066576,2.066576,2.066576,49.295538,2.066576,2.066576,2.066576
4,STT5441_Breast_004_normal,normal,2.613616,3.434965,12.682222,10.543189,26.688686,12.484822,1.364917,67.140393,...,11.227770,1.364917,1.364917,1.364917,1.364917,1.364917,23.627911,1.364917,1.364917,1.364917


In [27]:
def write_ans(name, k):
    with open(name, 'w') as f:
        f.write(str(k))

In [7]:
data_normal = data.loc[data['Diagnosis'] == 'normal'].drop(columns=['Diagnosis', 'Patient_id'])
data_early = data.loc[data['Diagnosis'] == 'early neoplasia'].drop(columns=['Diagnosis', 'Patient_id'])
data_cancer = data.loc[data['Diagnosis'] == 'cancer'].drop(columns=['Diagnosis', 'Patient_id'])

In [30]:
k_normal = 0
k_early = 0
alpha = 0.05
for i in data_normal.columns:
    if stats.ttest_ind(data_normal[i].values, data_early[i].values, equal_var=False)[1] < alpha:
        k_normal += 1
    if stats.ttest_ind(data_early[i].values, data_cancer[i].values, equal_var=False)[1] < alpha:
        k_early += 1
print(k_normal, k_early)
write_ans('1-1.txt', k_normal)
write_ans('1-2.txt', k_early)

1575 3490


In [21]:
import statsmodels.stats.multitest as smm

In [36]:
columns = data_normal.columns
normal_early = [stats.ttest_ind(data_normal[i].values, data_early[i].values, equal_var=False)[1] for i in columns]
early_cancer = [stats.ttest_ind(data_early[i].values, data_cancer[i].values, equal_var=False)[1] for i in columns]

In [23]:
def fold_change(c, t):
    return t/c if t > c else -c/t

In [37]:
m1 = smm.multipletests(normal_early, alpha=0.025, method='holm')[0]
m2 = smm.multipletests(early_cancer, alpha=0.025, method='holm')[0]

In [41]:
def practical_diff(m, data1, data2):
    global columns
    k = 0
    for i in range(len(m)):
        if m[i] and np.abs(fold_change(np.mean(data1[columns[i]].values), np.mean(data2[columns[i]].values))) > 1.5:
            k += 1
    return k

k_2_1 = practical_diff(m1, data_normal, data_early)
k_2_2 = practical_diff(m2, data_early, data_cancer)
print(k_2_1, k_2_2)
write_ans('2-1.txt', k_2_1)
write_ans('2-2.txt', k_2_2)

2 77


In [43]:
m3 = smm.multipletests(normal_early, alpha=0.025, method='fdr_bh')[0]
m4 = smm.multipletests(early_cancer, alpha=0.025, method='fdr_bh')[0]

In [44]:
k_3_1 = practical_diff(m3, data_normal, data_early)
k_3_2 = practical_diff(m4, data_early, data_cancer)
print(k_3_1, k_3_2)
write_ans('3-1.txt', k_3_1)
write_ans('3-2.txt', k_3_2)

4 524
